In [1]:

"""
Created on Thu Sep 22 16:15:46 2022

@author: Youcef Tahari 

[Mandatory]
HW2: Try web scraping with Taiwan’s bike dataset
Description: Please use Taiwan’s bike dataset to draw scatter plot. You can decide which region, which kind of bike (the common one in Taiwan is Ubike) and the objective of the analysis.

I mapped the u-bikes avaiblable in Zhongshan district - Taipei 

"""


import pandas as pd
import requests
import json
import folium




In [2]:
# download the ubike data of Hsinchu from https://data.gov.tw/dataset/67781
response = requests.get("http://odws.hccg.gov.tw/001/Upload/25/OpenData/9059/59/67e94b18-b8ba-4c6a-9a2a-e66638a31895.json")
content = response.content
json_tree = json.loads(content)

# data precess
data = pd.DataFrame(json_tree)
data = data.rename(columns={"站點名稱": "Site Name", 
                   "經度": "Longitude", 
                   "緯度": "Latitude", 
                   "圖片URL": "URL", 
                   "站點位置": "Site Address"})
# data : 63 rows × 5 columns

data
# Set the ceter of our map : NYCU managemant building 
# hsinchu_coords = [24.8138, 120.9675]
# NYCU Location: [24.787178345630263, 120.99748615357079]

Location_coords = [24.7871, 120.9974]
#Create the map
my_map = folium.Map(location = Location_coords, zoom_start = 13)

for i in range(0, 63):
  folium.Marker([data['Latitude'][i], data['Longitude'][i]], 
                popup = "Site name: " + data['Site Name'][i]). add_to(my_map)
my_map

In [3]:
# Taipei Youbike 

response = requests.get("https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json")
content = response.content
json_tree = json.loads(content)
#pprint.pprint(json_tree[1])
# sno：站點代號、 sna：場站名稱(中文)、 tot：場站總停車格、 sbi：場站目前車輛數量、 sarea：場站區域(中文)、 mday：資料更新時間、 
# lat：緯度、 lng：經度、 
# ar：地(中文)、 sareaen：場站區域(英文)、 snaen：場站名稱(英文)、 aren：地址(英文)、 bemp：空位數量、 act：全站禁用狀態

data = pd.DataFrame(json_tree, columns = ['sna', 'lng', 'lat', 'tot','sbi','bemp','act'])
data = data.rename(columns={"sna": "SiteName", "lng": "Longitude", "lat": "Latitude", "sbi": "Remaining"})
# data : 1168 rows × 3 columns

data

,SiteName,Longitude,Latitude,tot,Remaining,bemp,act
0,YouBike2.0_捷運科技大樓站,121.54360,25.02605,28,2,26,1
1,YouBike2.0_復興南路二段273號前,121.54357,25.02565,21,0,21,1
2,YouBike2.0_國北教大實小東側門,121.54124,25.02429,16,3,13,1
3,YouBike2.0_和平公園東側,121.54282,25.02351,11,5,6,1
4,YouBike2.0_辛亥復興路口西北側,121.54299,25.02153,16,15,1,1
...,...,...,...,...,...,...,...
1206,YouBike2.0_臺大總圖書館西南側,121.54031,25.01690,30,29,0,1
1207,YouBike2.0_臺大黑森林西側,121.54347,25.01995,20,18,1,1
1208,YouBike2.0_臺大獸醫館南側,121.54242,25.01791,24,20,4,1
1209,YouBike2.0_臺大新體育館東南側,121.53591,25.02112,40,40,0,1


In [4]:
# Define coordinates of where we want to center our map
# hsinchu_coords = [24.8138, 120.9675]
# NYCU Location: [24.787178345630263, 120.99748615357079]
# 台大管院 = [25.014138089260964, 121.5379950959105]
#Zhongshan district [25.0526981, 121.5203977 ]
Location_coords = [25.0526981, 121.5203977 ] 
#Create the map
my_map = folium.Map(location = Location_coords, zoom_start = 17)

for i in range(0, 1168):
  folium.Marker([data['Latitude'][i], data['Longitude'][i]], 
                popup = "Site name: " + data['SiteName'][i]). add_to(my_map)

my_map

In [9]:
# Ramaining bicycles around me 

my_map = folium.Map(location = Location_coords, zoom_start = 16.4, tiles='Stamen Toner')

# data = data[(data.lat > 25.01) & (data.lat < 25.02) & (data.lng > 121.53) & (data.lng < 121.54) ]
data = data[data.Remaining > 0]

from folium import plugins
bike = plugins.MarkerCluster().add_to(my_map)

for lat, lng, label, in zip(data.Latitude, data.Longitude, data.Remaining):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', prefix='fa', icon='bicycle'),
        popup=label,
    ).add_to(bike)
# add incidents to map

folium.Circle(radius=500, location=[25.0141, 121.5379], popup='NTU Finance', color='crimson', fill=True,).add_to(my_map)
folium.Marker([25.0526981, 121.5203977 ], popup='NTU Finance', 
              icon=folium.Icon(color='red', prefix='fa', icon='user')).add_to(my_map)

my_map.add_child(bike)
my_map.save('mymap.html')
from html2image import Html2Image
hti = Html2Image()
hti.screenshot(
    html_file='mymap.html',
    save_as='Bike_map.png'
)



[1101/134319.961746:ERROR:command_buffer_proxy_impl.cc(128)] ContextResult::kTransientFailure: Failed to send GpuControl.CreateCommandBuffer.
[1101/134325.338137:INFO:headless_shell.cc(660)] Written to file /Users/youcefnazimtahari/Documents/Special topics in financial engineering /DEDA/HW2/HW2_TracyZhou_0453946_YoubikeLocation/Bike_map.png.


['/Users/youcefnazimtahari/Documents/Special topics in financial engineering /DEDA/HW2/HW2_TracyZhou_0453946_YoubikeLocation/Bike_map.png']

In [8]:
!pip install html2image